In [1]:
import pandas as pd


### Import File

* Import the data file, and make sure you've imported it correctly!

In [2]:
file = "Resources/purchase_data.csv"
data = pd.read_csv(file)
df = pd.DataFrame(data)

df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


### Player Count

* Total Number of Players

In [3]:
# Find the number of unique SNs
totalPlayers = df["SN"].nunique()

# Create a DF and make a column header so it looks nice
totalPlayers_df = pd.DataFrame([totalPlayers])
totalPlayers_df.columns = ["Total Players"]
totalPlayers_df

,Total Players
0,576


### Purchasing Analysis (Total)

* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue

In [4]:
# Create variables 
nuniqueItems = df["Item ID"].nunique()
totPurch = len(df)
avgPric = df["Price"].mean()
totRev = df["Price"].sum()
# Put them into a DF
purchAnalysis_df = pd.DataFrame({
    "Unique Items":nuniqueItems,
    "Avg Price":avgPric,
    "Total Purchases":totPurch,
    "Total Revenue":totRev
}, index=[0])

# Format the values
purchAnalysis_df["Avg Price"] = purchAnalysis_df["Avg Price"].map("${:.2f}".format)
purchAnalysis_df["Total Revenue"] = purchAnalysis_df["Total Revenue"].map("${:.2f}".format)
purchAnalysis_df

,Unique Items,Avg Price,Total Purchases,Total Revenue
0,179,$3.05,780,$2379.77


### Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [5]:
# Pull only the SN and Gender columns
gender_df = df.loc[:, ["SN", "Gender",]]
# Take out any duplicates
gender_df = gender_df.drop_duplicates(["SN"])
# Group by gender, and count up the number of players, and put it in decending order
gender_df = gender_df.groupby(["Gender"], as_index=False).agg({"SN" : "count"})
gender_df = gender_df.sort_values("SN", ascending=False).reset_index(drop=True)
gender_df = gender_df.rename(columns = {"SN": "Total Count"})
# Calculate the percentage of players
gender_df["Percentage of Players"] = (gender_df["Total Count"] / totalPlayers)

# Format the values
gender_df["Percentage of Players"] = gender_df["Percentage of Players"].map("{:.2%}".format)
gender_df

,Gender,Total Count,Percentage of Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other / Non-Disclosed,11,1.91%


### Purchasing Analysis (Gender)

* The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Gender

In [6]:
# Create a new dataframe, drop any unnecessary columns, add any that will be needed. 
purchGender_df = df.copy()
purchGender_df = purchGender_df.drop(["Purchase ID", "Age", "Item ID", "Item Name"], axis = 1)
purchGender_df["AvgPurc"] = purchGender_df["Price"]
# Group by gender, and perform any calculations up front. 
purchGender_df = purchGender_df.groupby(["Gender"], as_index=False).agg({"SN":"count", "Price":"sum", "AvgPurc":"mean"})
# Rename columns
purchGender_df = purchGender_df.rename(columns= {"SN":"Purchase Count", "Price":"Total Purchase Value", "AvgPurc":"Avg Purchase Price"})

# Split up values by gender
columns = ["SN", "Price"]
male_df = df.loc[df["Gender"] == "Male"]
male_df = male_df[columns]
female_df = df.loc[df["Gender"] == "Female"]
female_df = female_df[columns]
other_df = df.loc[df["Gender"] == "Other / Non-Disclosed"]
other_df = other_df[columns]

# Calculate the AVG per person by gender
malePerPerson = male_df.groupby(["SN"]).sum().mean()
femalePerPerson = female_df.groupby(["SN"]).sum().mean()
otherPerPerson = other_df.groupby(["SN"]).sum().mean()
# Place the values into a list
purchPerUsr = pd.DataFrame({"Avg Purchase per Person": [femalePerPerson[0], malePerPerson[0], otherPerPerson[0]]})
# Assign the values to a column in our DF, and sort in descending order
purchGender_df["Avg Purchase per Person"] = purchPerUsr
purchGender_df = purchGender_df.sort_values("Purchase Count", ascending=False).reset_index(drop=True)

# Format the values
purchGender_df["Avg Purchase Price"] = purchGender_df["Avg Purchase Price"].map("${:.2f}".format)
purchGender_df["Total Purchase Value"] = purchGender_df["Total Purchase Value"].map("${:.2f}".format)
purchGender_df["Avg Purchase per Person"] = purchGender_df["Avg Purchase per Person"].map("${:.2f}".format)

purchGender_df

,Gender,Purchase Count,Total Purchase Value,Avg Purchase Price,Avg Purchase per Person
0,Male,652,$1967.64,$3.02,$4.07
1,Female,113,$361.94,$3.20,$4.47
2,Other / Non-Disclosed,15,$50.19,$3.35,$4.56


## Age Demographics
* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table

In [7]:
# Create the bins
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Create a new DF, drop the duplicates, rename SN
age_df = df.loc[:, ["SN", "Age",]].drop_duplicates(["SN"]).rename(columns = {"SN":"Total Count"})

# Group by the age range, count players, calculate percentage
age_df["Age Range"] = pd.cut(age_df["Age"], bins, labels=group_names, include_lowest=True)
age_df = age_df.groupby(["Age Range"], as_index=True).agg({"Total Count" : "count"})
age_df["Percentage of Players"] = (age_df["Total Count"] / totalPlayers)

#Format values
age_df["Percentage of Players"] = (age_df["Percentage of Players"]).map("{:.2%}".format)

age_df

,Total Count,Percentage of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


### Purchasing Analysis (Age)

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group


In [8]:
# Create bins
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
# Create our new DF
agePurch_df = df.copy()
# Create the necessary columns, drop the rest. 
agePurch_df["Age Range"] = pd.cut(agePurch_df["Age"], bins, labels=group_names, include_lowest=True)
agePurch_df = agePurch_df.drop(["Purchase ID", "Age", "Item ID", "Item Name"], axis = 1)
agePurch_df["AvgPurc"] = agePurch_df["Price"]
agePurch_df["Avg Total Purchase per Person"] = agePurch_df["Price"]

# Create a copy of the DF we have so far. Create a list. 
perTotAge_df = agePurch_df.copy()
perTotAge = []
# For each name in groupnames, group purchases by SN, add them and then average them. 
# Then place them into a list. 
for x in range(len(group_names)):
    ageTemp_df = perTotAge_df.loc[perTotAge_df["Age Range"] == group_names[x]]
    bracketAvgAge = ageTemp_df.groupby(["SN"]).sum().mean()
    perTotAge.append(bracketAvgAge["Avg Total Purchase per Person"])

# Group agePurch_df by the age range, and perform necessary calculations. Also rename columns.
agePurch_df = agePurch_df.groupby(["Age Range"], as_index=True).agg({"SN":"count", "Price":"sum", "AvgPurc":"mean"})
agePurch_df = agePurch_df.rename(columns= {"SN":"Purchase Count", "Price":"Total Purchase Value", "AvgPurc":"Avg Purchase Price"})
# Set the total per person to the list made previously 
agePurch_df["Avg Total Purchase per Person"] = perTotAge

# Format the values. 
agePurch_df["Total Purchase Value"] = agePurch_df["Total Purchase Value"].map("${:.2f}".format)
agePurch_df["Avg Purchase Price"] = agePurch_df["Avg Purchase Price"].map("${:.2f}".format)
agePurch_df["Avg Total Purchase per Person"] = agePurch_df["Avg Total Purchase per Person"].map("${:.2f}".format)
agePurch_df



,Purchase Count,Total Purchase Value,Avg Purchase Price,Avg Total Purchase per Person
Age Range,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,$1114.06,$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
40+,13,$38.24,$2.94,$3.19


### Top Spenders

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value


In [9]:
# Create a new DF, rename the columns, and add a new one for Avg price
topSpend_df = df.loc[:, ["SN", "Price", "Item ID"]].rename(columns = {"Price":"Total Purchase Value","Item ID":"Purchase Count"})
topSpend_df["Avg Purchase Price"] = topSpend_df["Total Purchase Value"]
# Group by the SN, do the calculations, and sort by the top spender
topSpend_df = topSpend_df.groupby(["SN"], as_index=True).agg({"Purchase Count":"count","Avg Purchase Price":"mean", "Total Purchase Value":"sum"})
topSpend_df = topSpend_df.sort_values("Total Purchase Value", ascending=False)

# Format the values
topSpend_df["Avg Purchase Price"] = topSpend_df["Avg Purchase Price"].map("${:.2f}".format)
topSpend_df["Total Purchase Value"] = topSpend_df["Total Purchase Value"].map("${:.2f}".format)
# Save only the top 5
topSpend_df = topSpend_df.head()
topSpend_df

,Purchase Count,Avg Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


### Most Popular Items

* Identify the 5 most popular items by purchase count, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [46]:
# Create the ne DF, create the necessary columns
popItem_df = df.loc[:, ["Item ID", "Item Name", "Price"]]
popItem_df["Purchase Count"] = popItem_df["Item Name"]
popItem_df["Total Purchase Value"] = popItem_df["Price"]
# Group by the item Id, and keep the necessary columns. Also, do calculations
popItem_df = popItem_df.groupby(["Item ID", "Item Name", "Price"], as_index=False).agg({"Total Purchase Value":"sum", "Purchase Count":"count"})
# Sort by Purchase count
popItem_df = popItem_df.sort_values("Purchase Count", ascending=False)
# Save the top 5
popItem_df = popItem_df.head().reset_index(drop=True)

# Format the Values
popItem_df["Price"] = popItem_df["Price"].map("${:.2f}".format)
popItem_df["Total Purchase Value"] = popItem_df["Total Purchase Value"].map("${:.2f}".format)
popItem_df

,Item ID,Item Name,Price,Total Purchase Value,Purchase Count
0,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,$50.76,12
1,108,"Extraction, Quickblade Of Trembling Hands",$3.53,$31.77,9
2,145,Fiery Glass Crusader,$4.58,$41.22,9
3,82,Nirvana,$4.90,$44.10,9
4,37,"Shadow Strike, Glory of Ending Hope",$3.16,$25.28,8


### Most Profitable Items

* Identify the 5 most profitable items by total purchase value, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [57]:
# Create the ne DF, create the necessary columns
profItem_df = df.loc[:, ["Item ID", "Item Name", "Price"]]
profItem_df["Purchase Count"] = profItem_df["Item Name"]
profItem_df["Total Purchase Value"] = profItem_df["Price"]
# Group by the item Id, and keep the necessary columns. Also, do calculations
profItem_df = profItem_df.groupby(["Item ID", "Item Name", "Price"], as_index=False).agg({"Total Purchase Value":"sum", "Purchase Count":"count"})
# Sort by Purchase Value
profItem_df = profItem_df.sort_values("Total Purchase Value", ascending=False)
# Save the top 5
profItem_df = profItem_df.head().reset_index(drop=True)

# Format the Values
profItem_df["Price"] = profItem_df["Price"].map("${:.2f}".format)
profItem_df["Total Purchase Value"] = profItem_df["Total Purchase Value"].map("${:.2f}".format)
profItem_df

,Item ID,Item Name,Price,Total Purchase Value,Purchase Count
0,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,$50.76,12
1,82,Nirvana,$4.90,$44.10,9
2,145,Fiery Glass Crusader,$4.58,$41.22,9
3,92,Final Critic,$4.88,$39.04,8
4,103,Singed Scalpel,$4.35,$34.80,8
